Kernel methods are widespread in machine learning and they were particularly common before deep learning became a dominant paradigm. The core idea is to introduce a new notion of distance between high-dimensional data points by replacing the inner product $(x_i, x_j)$ by a function that retains many properties of the inner product, yet which is nonlinear. This function $k(.,.)$ is called a kernel. Then, in many cases, wherever a learning algorithm would use an inner product, the kernel function is used instead.

The intuition is that the kernel function acts as an inner product on a higher dimensional space and encompasses some $\phi(.)$ mapping from the original space of the data points to this space. So intuitively, the kernel function is $k(x_i, x_j)=(\phi(x_i), \phi(x_j))$. The hope is that points that were not linearly separable in the original space become linearly separable in the higher dimensional space. The $\phi(.)$ function may map to an infinite dimensional space and it does not actually have to be specified. As long as the kernel function is positive semidefinite, the idea works.

Many kernel-based learning algorithms are instance-based, which means that the final model retains some or all of the training instances and they play a role in the actual prediction. Support vector machines belong here: support vectors are the training instances which are critically important in defining the boundary between two classes. Some important kernels are listed below.

| Name | &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Kernel function|
|------|-----------------|
|Linear | $(x_i,x_j)$|
|Polynomial| $((x_i,x_j)+c)^d$|
|Radial basis function|$\exp(-\gamma\|x_i-x_j\|^2)$|

The choice of kernel and the parameters of the kernel are often arbitrary and either some trial and error on the dataset or hyperparameter optimization helps choose the right combination. Quantum computers naturally give rise to certain kernels and it is worth looking at a specific variant of how it is constructed.

 # Thinking backward: learning methods based on what the hardware can do

Instead of twisting a machine learning algorithm until it only contains subroutines that have quantum variants, we can reverse our thinking and ask: given a piece of quantum hardware and its constraints, can we come up with a new learning method? For instance, interference is a very natural thing to do: we showed an option in the first notebook on quantum states, and it can also be done with a Hadamard gate. 
For this to work we need to encode both training and testvectors as amplitudes in a statevector built up out of four registers:

$|0\rangle_c|00..0\rangle_m|00..0\rangle_i|0\rangle_a$

The amplitude of such state will be equal to the value of a feature in a training vector or test vector. To do that we use four registers. The first is a single bit, acting as the ancilla ancilla (a), which will will code for either a training (a=0) or a testvector (a=1). The second register, in the notebook example a single bit, will code for the m-th training vector. The third register, in the notebook example also reduced to a single bit, codes for the i-th feature. Lastly the class bit (c) codes for class -1 (c=0), or 1 (c=1).
Hence, if after fully encoding all training data and test data into the state $|\psi>$ the state |1010> has coefficient 0.46 :

$|\psi\rangle\ = ....+ 0.46|1010\rangle +....$  ,

Then that implies that the second feature (i=1) of the first (m=0) training vector (a=0), which classifies as class 1 (c=1), has the value 0.46. Note, we assume both training vectors and test vector are normalized.

In a more general expression we can write for a fully encoded state (NB we arrange the order of the registers to be consistent with the code below):

$|\psi\rangle = \frac{1}{\sqrt{2M}}\sum_{m=0}^{M-1}|y_m\rangle|m\rangle|\psi_{x^m}\rangle|0\rangle + |y_m\rangle|m\rangle|\psi_{\tilde{x}}\rangle|1\rangle$

with:

$|\psi_{x^m}\rangle = \sum_{i=0}^{N-1}x_i^m|i\rangle, \; |\psi_{\tilde{x}}\rangle = \sum_{i=0}^{N-1}\tilde{x_i}|i\rangle. \quad$ N being equal to the number of features in the the training and test vectors

As the last summation is independent on m, there will M copies of the test vector in the statevector, one for every training vector.


We now only need to apply a Hadamard gate to the ancilla to interfere the test and training instances. Measuring and post-selecting on the ancilla gives rise to a kernel [[1](#1)].

Let's start with initializations:





In [ ]:
from pyquil import Program, get_qc
from pyquil.gates import *
from forest_tools import *
from pyquil.api import WavefunctionSimulator
from pyquil.parameters import Parameter, quil_sin, quil_cos
from pyquil.quilbase import DefGate

np.set_printoptions(precision = 3)
%matplotlib inline

qvm_server, quilc_server, fc = init_qvm_and_quilc('/home/local/bin/qvm', '/home/local/bin/quilc')
qc = get_qc('4q-qvm', connection=fc)


Forest allows building user-defined gates, and we'll take advantage of this feature to define controlled rotations which will be used later to apply the rotations required to load the data vectors.

In [ ]:
# Define custom gates
# Controlled Ry
theta = Parameter('theta')
cry = np.array([
    [1, 0, 0, 0],
    [0, 1, 0, 0],
    [0, 0, quil_cos(theta / 2), -quil_sin(theta / 2)],
    [0, 0, quil_sin(theta / 2), quil_cos(theta / 2)]
])
cry_definition = DefGate('CRY', cry, [theta])
CRY = cry_definition.get_constructor()

# Double controlled Ry
alpha = Parameter('alpha')
ccry = np.array([
    [1, 0, 0, 0, 0, 0, 0, 0],
    [0, 1, 0, 0, 0, 0, 0, 0],
    [0, 0, 1, 0, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 0, 1, 0, 0],
    [0, 0, 0, 0, 0, 0, quil_cos(alpha / 2), -quil_sin(alpha / 2)],
    [0, 0, 0, 0, 0, 0, quil_sin(alpha / 2), quil_cos(alpha / 2)]
])
ccry_definition = DefGate('CCRY', ccry,[alpha])
CCRY = ccry_definition.get_constructor()


We are constructing an instance-based classifier: we will calculate a kernel between all training instances and a test example. In this sense, this learning algorithm is lazy: no actual learning happens and each prediction includes the entire training set.

As a consequence, state preparation is critical to this protocol. We have to encode the training instances in a superposition in a register, and the test instances in another register. Consider the following training instances of the [Iris dataset](https://archive.ics.uci.edu/ml/datasets/iris): $S = \{(\begin{bmatrix}0 \\ 1\end{bmatrix}, 0), (\begin{bmatrix}0.78861006 \\ 0.61489363\end{bmatrix}, 1)\}$, that is, one example from class 0 and one example from class 1. Furthermore, let's have two test instances, $\{\begin{bmatrix}-0.549\\ 0.836\end{bmatrix}, \begin{bmatrix}0.053 \\ 0.999\end{bmatrix}\}$. These examples were cherry-picked because they are relatively straightforward to prepare.

In [ ]:
training_set = [[0, 1], [0.78861006, 0.61489363]]
labels = [0, 1]
test_set = [[-0.549, 0.836], [0.053 , 0.999]]

To load the data vectors, we use amplitude encoding as explained above, which means that, for instance, the second training vector will be encoded as $0.78861006|0\rangle + 0.61489363|1\rangle$. Preparing these vectors only needs a rotation, and we only need to specify the corresponding angles. The first element of the training set does not even need that: it is just the $|1\rangle$ state, so we don't specify an angle for it.

To get the angle we need to solve the equation $a|0\rangle + b|1\rangle=\cos\left(\frac{\theta}{2}\right)|0\rangle + i \sin \left(\frac{\theta}{2}\right) |1\rangle$. Therefore, we will use $\theta=2 \arccos(a)$

In [ ]:
def get_angle(amplitude_0):
    return 2*np.arccos(amplitude_0)

In practice, the state preparation procedure we will consider requires the application of rotations in order to prepare each data point in the good register (see circuit below). Don't hesitate to check it by yourself by running the circuit below with a pen and paper.

In [ ]:
test_angles = [get_angle(test_set[0][0]), get_angle(test_set[1][0])]
training_angle = get_angle(training_set[1][0])

The following function builds the circuit. We plot it and explain it in more details below.

In [ ]:
def prepare_state(angles):
    ancilla_qubit = 0
    index_qubit = 1
    data_qubit = 2
    class_qubit = 3
    circuit = Program()
    circuit += cry_definition
    circuit += ccry_definition
    
    # Put the ancilla and the index qubits into uniform superposition
    circuit += H(ancilla_qubit)
    circuit += H(index_qubit)
    
    # Prepare the test vector
    circuit += CRY(angles[0])(ancilla_qubit, data_qubit)
    # Flip the ancilla qubit > this moves the input 
    # vector to the |0> state of the ancilla
    circuit += X(ancilla_qubit)
    
    # Prepare the first training vector
    # [0,1] -> class 0
    # We can prepare this with a Toffoli
    circuit += CCNOT(ancilla_qubit, index_qubit, data_qubit)
    # Flip the index qubit > moves the first training vector to the 
    # |0> state of the index qubit
    circuit += X(index_qubit)
    
    # Prepare the second training vector
    # [0.78861, 0.61489] -> class 1
    circuit += CCRY(angles[1])(ancilla_qubit, index_qubit, data_qubit)
   
    # Flip the class label for training vector #2
    circuit += CNOT(index_qubit, class_qubit)
    
    return circuit

Let's dissect the last part where we prepare the second trainig state, which is $\begin{pmatrix}0.78861 \\ 0.61489\end{pmatrix}$ and we entangle it with the excited state of the ancilla and the excited of the index qubit. We use `angles[1]`, which is `1.3245021469658966`. Why? We have to rotate the basis state $|0\rangle$ to contain the vector we want. We could write this generic state as $\begin{pmatrix} \cos(\theta/2) \\ \sin(\theta/2)\end{pmatrix}$. Looking at the documentation of the gate implementing the rotation, you'll see that the function argument divides the angle by two, so we have to adjust for that -- this is why we divided $\theta$ by two. If you calculate the arccos or arcsin values, you will get the value in `angles[1]`.

Let us see the circuit for the distance-based classifier:

In [ ]:
angles = [test_angles[0], training_angle]
state_preparation_0 = prepare_state(angles)
plot_circuit(state_preparation_0)

In the first section, the ancilla and index qubits are put into uniform superposition.

The second section entangles the test vector with the ground state of the ancilla.

In the third section, we prepare the state $|1\rangle$, which is the first training instance, and entangle it with the excited state of the ancilla and the ground state of the index qubit with a Toffoli gate and a Pauli-X gate. The Toffoli gate is also called the controlled-controlled-not gate, describing its action.

The fourth section prepares the second training instance and entangles it with the excited state of the ancilla and the index qubit. Also in this section, the class qubit is flipped conditioned on the index qubit being $|1\rangle$. This creates the connection between the encoded training instances and the corresponding class label.

Let's dissect the last part where we prepare the second training state, which is $\begin{pmatrix}0.790 \\ 0.615\end{pmatrix}$ and we entangle it with the excited state of the ancilla and the excited state of the index qubit. We use `angles[1]`, which is ~`1.325/2`. Why? We have to rotate the basis state $|0\rangle$ to contain the vector we want. We could write this generic state as $\begin{pmatrix} \cos(\theta/2) \\ \sin(\theta/2)\end{pmatrix}$. Looking at the documentation of the gate implementing the rotation, you'll see that the function argument divides the angle by two, so we have to adjust for that -- this is why we divided $\theta$ by two. If you calculate the arccos or arcsin values, you will get the value in `angles[1]`.

Let's now see what final state the circuit has produced

In [ ]:
val = ['Xtest x','Xtest y','Xtrn0 y','Xtest x','Xtrn1 x','Xtest y','Xtrn1 y']
wfn = WavefunctionSimulator(connection=fc).wavefunction(state_preparation_0)
wfn_str = str(wfn).split(">")
print("Normalized state vector\n")
print("           cdia    coefficient")
for i in range(len(wfn_str)-1):
    a= wfn_str[i].split(")|")
    print(val[i],'  ', a[1],'  ', a[0][a[0].find("(")+1:])


From the table you can see how both the test vector (Xtst x, Xtsty), as well as the training vectors ((Xtrn0 x, Xtrn0 y) - class0) and ((Xtrn1 x,Xtrn1 y) - class1) are embedded in the state vector.(NB Xtrn1 does not appear as coeficient for $|0001\rangle$, as it is 0).<br>
The training vector class is indicated in the class bit (c). The test vector is coded by the 0-state of the ancilla (a), and the training vector is coded by the 1-state of the ancilla. Note also the data bit (d) coding for the value of the x or y feature of the training vectors, and the index bit (i) coding for training vector 1 or 2.

We are now ready for the final step

# A natural kernel on a shallow circuit

Having done the state preparation, the actual prediction is nothing but a Hadamard gate applied on the ancilla, followed by measurements. Since the ancilla is in a uniform superposition at the end of the state preparation and it is entangled with the registers encoding the test and training instances, applying a second Hadamard on the ancilla interferes the entangled registers. The state before the measurement is  $\frac{1}{2\sqrt{M}}\sum_{m=0}^{M-1}|y_m\rangle|m\rangle(|\psi_{x^m}\rangle+|\psi_{\tilde{x}}\rangle)|0\rangle+|y_m\rangle|m\rangle(|\psi_{x^m}\rangle-|\psi_{\tilde{x}}\rangle)|1\rangle$, where $|\psi_{\tilde{x}}\rangle$ is the encoded test instance and $\psi_{x^m}\rangle$ is the m-th training instance. For our example M, the number of training samples, equals 2.

In [ ]:
def interfere_data_and_test_instances(circuit, angles):
    ro = circuit.declare(name='ro', memory_type='BIT', memory_size=4)
    circuit += H(0)
    for q in range(4):
        circuit += MEASURE(q, ro[q])
        
    return circuit

If we measure the ancilla, the outcome probability of observing 0 will be $\frac{1}{4M}\sum_{i=0}^{M-1} |\tilde{x} + x_m|^2$. This creates a kernel of the following form:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
x = np.linspace(-2, 2, 100)
plt.xlim(-2, 2)
plt.ylim(0, 1.1)
plt.plot(x, 1-x**2/4)

This is the kernel that performs the classification. We perform the post-selection on observing 0 on the measurement on the ancilla and calculate the probabilities of the test instance belonging to either class:

In [ ]:
def postselect(result_counts):
    total_samples = sum(result_counts.values())

    # define lambda function that retrieves only results where the ancilla is in the |0> state
    post_select = lambda counts: [(state, occurences) for state, occurences in counts.items() if state[0] == '0']

    # perform the postselection
    postselection = dict(post_select(result_counts))
    postselected_samples = sum(postselection.values())

    print(f'Ancilla post-selection probability was found to be {postselected_samples/total_samples}')

    retrieve_class = lambda binary_class: [occurences for state, occurences in postselection.items() if state[-1] == str(binary_class)]

    prob_class0 = sum(retrieve_class(0))/postselected_samples
    prob_class1 = sum(retrieve_class(1))/postselected_samples

    print('Probability for class 0 is', prob_class0)
    print('Probability for class 1 is', prob_class1)

For the first instance we have:

In [ ]:
circuit_0 = interfere_data_and_test_instances(state_preparation_0, angles)

circuit_0.wrap_in_numshots_loop(1024)
executable = qc.compile(circuit_0)
measures = qc.run(executable)

count = np.unique(measures, return_counts=True, axis=0)
count = dict(zip(list(map(lambda l: ''.join(list(map(str, l))), count[0].tolist())), count[1]))

postselect(count)

And for the second one:

In [ ]:
angles = [test_angles[1], training_angle]
state_preparation_1 = prepare_state(angles)


In [ ]:
circuit_1 = interfere_data_and_test_instances(state_preparation_1, angles)
circuit_1.wrap_in_numshots_loop(1024)
executable = qc.compile(circuit_1)
measures = qc.run(executable)

count = np.unique(measures, return_counts=True, axis=0)
count = dict(zip(list(map(lambda l: ''.join(list(map(str, l))), count[0].tolist())), count[1]))

postselect(count)

In [ ]:
qvm_server.terminate()
quilc_server.terminate()

# References

[1] M. Schuld, M. Fingerhuth, F. Petruccione. (2017). [Implementing a distance-based classifier with a quantum interference circuit](https://doi.org/10.1209/0295-5075/119/60002). *Europhysics Letters*, 119(6), 60002. <a id='1'></a>